# Association/dissociation of NaCl

First, let's import BioSimSpace:

In [ ]:
import BioSimSpace as BSS

Next, let's load the solvated NaCl system:

In [ ]:
system = BSS.IO.readMolecules(["../input/nacl.gro", "../input/nacl.top"])

Search the system for the ions and get their index:

In [ ]:
na = system.getIndex(system.search("atomname NA")[0])
cl = system.getIndex(system.search("atomname CL")[0])

Create an upper bound at a distance of 0.6 nm:

In [ ]:
upper_bound = BSS.Metadynamics.Bound(0.6*BSS.Units.Length.nanometer, force_constant=2000)

Create a grid on which to sample the collective variable:

In [ ]:
grid = BSS.Metadynamics.Grid(minimum=0*BSS.Units.Length.nanometer, maximum=BSS.Units.Length.nanometer)

Create a collective variable representing the distance between the ions:

In [ ]:
colvar = BSS.Metadynamics.CollectiveVariable.Distance(atom0=na, atom1=cl,
    hill_width=0.02*BSS.Units.Length.nanometer,upper_bound=upper_bound, grid=grid)

Define the metadynamics protocol:

In [ ]:
protocol = BSS.Protocol.Metadynamics(colvar, bias_factor=5)

Initialise a process to run the metadynamics protocol:

In [ ]:
process = BSS.Metadynamics.run(system, protocol)

Plot the time evolution of the collective variable. (Note that this simulation is much slower than the alanine-dipeptide example, so it might take a while for data to appear.)

In [ ]:
BSS.Notebook.plot(process.getTime(time_series=True), process.getCollectiveVariable(0, time_series=True))

Plot the free energy estimate:

In [ ]:
free_nrgy = process.getFreeEnergy()
BSS.Notebook.plot(free_nrgy[0], free_nrgy[1])

Note that the free energy increases beyond 0.6 nm because of the upper bound that we've imposed.

Sample confugrations from the basin near 0.25 nm:

In [ ]:
bounds = [(0.225*BSS.Units.Length.nanometer, 0.275*BSS.Units.Length.nanometer)]
configs, colvars = process.sampleConfigurations(bounds, 20)

View the first configuration (ions only):

In [ ]:
config = configs[0]
config.removeWaterMolecules()
view = BSS.Notebook.View(config)
view.system()